This is an analysis of complaints data, munged [here](https://github.com/TheOregonian/long-term-care-db/blob/master/notebooks/transformation/mung-3-29-scrape.ipynb).

The fields are:
1. <i>abuse_number:</i> A unique number assigned each complaint.
2. <i>facility_id:</i> A unique number to each facility building. Stays if ownership changes.
3. <i>facility_type:</i> NF: Nursing Facility; ALF: Assisted Living Facility; RCF: Residential Care Facility.
4. <i>facility_name:</i> Name of facility as of January 2017, when DHS provided the facility data to The Oregonian.
5. <i>abuse_type:</i> A – facility abuse; L – licensing. Note: This does not apply to nursing facilities. All their complaints are either blank in this field or licensing.
6. <i>fine:</i> Amount that state initialy fined the facility. Not necessarily amount of final fine.
7. <i>action_notes:</i> DHS determination of what general acts constituted the abuse or rule violation.
8. <i>incident_date:</i> Date the incident occured
9. <i>outcome:</i> A very brief description of the consequences of the abuse or rule violation to the resident
10. <i>outcome_notes:</i> A detailed description of what happened.
11. <i>year:</i> year incident occured
12. <i>online_fac_name:</i> If complaint is online, name listed for the facility
13. <i>public:</i> Whether or not complaint is online
14. <i>omg_outcome:</i> Field we created to group some similar outcomes. 

In [17]:
import pandas as pd
import numpy as np
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
pd.set_option('display.max_colwidth', -1)

In [18]:
df = pd.read_csv('../../data/processed/complaints-3-29-scrape.csv')

<h3>How many total complaints are there?</h3>

In [19]:
df.count()[0]

13032

<h3>How many complaints do not appear in the state's public database?</h3>

In [20]:
df[df['public']=='offline'].count()[0]

7846

<h3>How many complaints do appear in the state's public database?</h3>

In [21]:
df[df['public']=='online'].count()[0]

5186

<h3>What percent of complaints are missing?</h3>

In [22]:
df[df['public']=='offline'].count()[0]/df.count()[0]*100

60.205647636586868

<h3>How many complaints were labelled 'Exposed to potential harm' or 'No negative outcome?'</h3>

In [23]:
df[(df['outcome']=='Exposed to Potential Harm') | (df['outcome']=='No Negative Outcome')].count()[0]

2509

<h3>Of all missing complaints, what percent are in the above two categories?</h3>

In [24]:
df[(df['outcome']=='Exposed to Potential Harm') |
   (df['outcome']=='No Negative Outcome')].count()[0]/df[df['public']=='offline'].count()[0]*100

31.978078001529443

<h3>What's the online/offline breakdown by outcome?</h3>
<i>This was used in graphics</i>

In [25]:
totals = df.groupby(['omg_outcome','public']).count()['abuse_number'].unstack().reset_index()

In [26]:
totals.fillna(0, inplace = True)

In [27]:
totals['total'] = totals['online']+totals['offline']

In [28]:
totals['pct_offline'] = round(totals['offline']/totals['total']*100)

In [29]:
totals.sort_values('pct_offline',ascending=False)

public                                        omg_outcome  offline  online  \
16      Staffing issues                                    12.0     0.0      
1       Denied readmission or moved improperly             35.0     2.0      
14      Potential harm                                     2361.0   148.0    
3       Fall, no injury                                    150.0    13.0     
8       Left facility without attendant, no injury         207.0    18.0     
9       Loss of Dignity                                    884.0    97.0     
12      Medication error                                   983.0    217.0    
5       Inadequate care                                    496.0    170.0    
6       Inadequate hygiene                                 138.0    104.0    
10      Loss of property, theft or financial exploitation  809.0    737.0    
13      Physical abuse                                     89.0     92.0     
18      Verbal or emotional abuse                          70.0     94.0     
7       Involuntary seclusion                              8.0      11.0     
2       Failure to address resident aggression             395.0    622.0    
4       Fracture or other injury                           680.0    1185.0   
11      Medical condition developed or worsened            370.0    1046.0   
0       Death                                              7.0      23.0     
15      Sexual abuse                                       15.0     49.0     
17      Unreasonable discomfort or continued pain          115.0    452.0    
19      Weight loss                                        20.0     106.0    

public   total  pct_offline  
16      12.0    100.0        
1       37.0    95.0         
14      2509.0  94.0         
3       163.0   92.0         
8       225.0   92.0         
9       981.0   90.0         
12      1200.0  82.0         
5       666.0   74.0         
6       242.0   57.0         
10      1546.0  52.0         
13      181.0   49.0         
18      164.0   43.0         
7       19.0    42.0         
2       1017.0  39.0         
4       1865.0  36.0         
11      1416.0  26.0         
0       30.0    23.0         
15      64.0    23.0         
17      567.0   20.0         
19      126.0   16.0

<h3>How many offline complaints in the database were found to have "abuse," "neglect" or "exploitation?"</h3>

In [30]:
df['outcome_notes'].fillna('', inplace = True)

In [31]:
df[(df['outcome_notes'].str.contains('constitute neglect|constitutes neglect|constitute abuse|constitutes abuse|constitutes exploitation|constitutes financial exploitation')) & (df['public']=='offline')].count()[0]

483

"The state fined the facilities in hundreds of those cases."

<h3>In how many 'potential harm' cases were facilities fined?</h3>

In [32]:
df[(df['omg_outcome']=='Potential harm') & (df['fine']>0) & (df['public']=='offline')].count()[0]

206